# Roberta - masked language modelling - Rupert AI

The Microsoft Research Sentence Completion Challenge - MRSCC (Zweig and Burges, 2011) requires a system to be able to predict which is the most likely word (from a set of 5 possibilities) to complete a sentence.

Roberta is trained on a masked language modelling task so it can complete this task straight out of the box! However, to get the best performance we should match our training distribution to our target distribution (sherlock holmes) - a great example of transfer learning.

* Training data: a training corpus of 19th century novels data (522 files) from Project Gutenberg data.
* Test data: 1040 sentences from sherlock holmes novels with one missing word and 5 options to choose from

In [28]:
%%capture
! pip install -U accelerate
! pip install -U transformers
!pip install datasets

# Data processing

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import os
mrscc_dir = '/content/drive/MyDrive/sentence-completion'
questions = pd.read_csv(os.path.join(mrscc_dir,"testing_data.csv"))
answers = pd.read_csv(os.path.join(mrscc_dir,"test_answer.csv"))

# Retrain Masked Language Modelling Model!

Retrain a RoBERTa model on a masked language modelling task.

In [31]:
def processfiles(files, train_dir):
  texts = []
  for i, a_file in enumerate(files):
    text = ""
    try:
      with open(os.path.join(train_dir, a_file)) as instream:
        for line in instream:
          text += line
        texts.append(text)
    except UnicodeDecodeError:
      print(f"Unicode error for this file {a_file}")
  return texts

In [32]:
import glob
file_names = glob.glob('/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/*')
file_names

['/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/1DFRE10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/3ELPH10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/6DFRE10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/AESOPA10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/4DFRE10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/2000010.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/3BOAT10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/19TOM10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/2MROW10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/04TOM10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/2DINA10.TXT',
 '/content/drive/MyDrive/sentence-completion/Holmes_Training_Data/AESOP11.TXT',
 '/content/drive/MyDrive/sentence-compl

In [27]:
texts = processfiles(file_names, mrscc_dir)


Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/ACHOE10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/HHOHG10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/DTROY10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/HFDTR10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/GHROS10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/LLIFE10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/KRSON10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/PHIL410.TXT
Unicode error for this file /content/drive/MyDrive/sentence-completion/Holmes_Training_Data/MOHIC10.TXT
Unicode error for this file /content/drive/MyDrive/sentence-comp

Creating hugging face dataset from text files

In [33]:
from transformers import AutoTokenizer
import datasets
from datasets import Dataset

In [34]:
ds_len = 128
texts_dict = {'text': [t for t in texts[:ds_len]]}

In [35]:
ds = Dataset.from_dict(texts_dict)

In [36]:
ds

Dataset({
    features: ['text'],
    num_rows: 128
})

In [37]:
ds = ds.train_test_split(test_size=0.1)

In [38]:
model_name = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenised_ds = ds.map(lambda batch: tokenizer(batch['text']), batched=True, num_proc=2, remove_columns=['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/115 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (67163 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (160106 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=2):   0%|          | 0/13 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (20899 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (269235 > 512). Running this sequence through the model will result in indexing errors


In [39]:
block_size = 128

def group_texts(b):
  concat_text = {k: sum(b[k], []) for k in b.keys()}
  total_length = len(concat_text[list(b.keys())[0]])
  total_length = (total_length // block_size) * block_size
  result = {k: [t[i: i + block_size] for i in range(0, total_length, block_size)] for k, t in concat_text.items()}
  result['labels'] = result['input_ids'].copy()
  return result

In [40]:
blocked_ds = tokenised_ds.map(group_texts, batched=True, num_proc=2, batch_size=1000)

Map (num_proc=2):   0%|          | 0/115 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/13 [00:00<?, ? examples/s]

In [41]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [42]:
tokenizer.convert_ids_to_tokens(50264)

'<mask>'

create hugging face trainer

In [45]:
from transformers import AutoModelForMaskedLM, Trainer, TrainingArguments
trainer_model = AutoModelForMaskedLM.from_pretrained(model_name, return_dict=True)

train_args = TrainingArguments("test-mlm",
                               evaluation_strategy='epoch',

                               learning_rate=2e-5,
                               weight_decay=0.01,
                               num_train_epochs=1,
                               per_device_train_batch_size = 32,
                               save_steps=20000)


trainer = Trainer(model=trainer_model, args=train_args, train_dataset=blocked_ds['train'], eval_dataset=blocked_ds['test'], data_collator=data_collator)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,2.295700,2.159750


TrainOutput(global_step=3399, training_loss=2.3621888122827945, metrics={'train_runtime': 2507.5038, 'train_samples_per_second': 43.373, 'train_steps_per_second': 1.356, 'total_flos': 3605920610333184.0, 'train_loss': 2.3621888122827945, 'epoch': 1.0})

# Apply model to MLM task

N.B. Each candidate word is placed into each sentence - resulting in 5 candidate sentences. Each of these sentences are then tokenised and all candidate word **tokens** are masked. E.g. the flowers were placed *instantaenously* on the table will have several < mask > tokens as the token ID of *instantaenously* consists of several token IDs.

In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import numpy as np
import re

choices = {'a)':1, 'b)':2, 'c)':3, 'd)':4, 'e)':5}
model_name = 'distilroberta-base'


In [48]:
class LanguageModelEvaluator():

  def __init__(self, q, a, c, mn, model):
    self.questions, self.answers, self.choices, self.model_name = q, a, c, mn
    print(len(self.questions))
    self.process_questions_and_answers()
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.model = model
    self.sent_encodings, self.word_encodings, self.mask_idxs = self.make_encodings()

  def run_model_and_evaluate(self):
    output = self.make_predictions()
    self.accuracy = self.get_model_accuracy(output, self.questions['answer'])

  def process_questions_and_answers(self, s='_____'):
    answer_idxs, candidate_questions = [], []
    for index, row in self.questions.iterrows():
      answer = answers.iloc[index].answer + ')'
      answer_idxs.append(self.choices.get(answer))
      candidate_questions.append([re.sub(s, row.loc[c], row.loc['question']) for c in self.choices.keys()])
    self.questions.loc[:, 'candidate_questions'] = candidate_questions
    self.questions.loc[:, 'answer'] = answer_idxs

  def get_sublist_idxs_in_list(self, word, sentence):
    # find mask indicies for encoded sentence
    possibles = np.where(sentence == word[0])[0]
    for p in possibles:
      check = sentence[p:p + len(word)]
      if np.all(check == word):
          return list(range(p, (p + len(word))))

  def make_encodings(self):
    sent_encodings, word_encodings, mask_idxs = [], [], []
    for index, row in self.questions.iterrows():
        _sent_encodings, _word_encodings, _mask_idxs = [], [], []
        for i, (word, sentence) in enumerate(zip(row[self.choices.keys()], row.loc['candidate_questions'])):
          encoded_word = self.tokenizer.encode(str(" " + word), add_special_tokens=False)
          encoded_sent = self.tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='pt',
                                                padding='max_length', max_length=128, return_attention_mask=True)
          tokens_to_mask_idx = self.get_sublist_idxs_in_list(np.array(encoded_word), np.array(encoded_sent['input_ids'][0]))
          encoded_sent['input_ids'][0][tokens_to_mask_idx] = self.tokenizer.mask_token_id
          _sent_encodings.append(encoded_sent)
          _word_encodings.append(encoded_word)
          _mask_idxs.append(tokens_to_mask_idx)
        sent_encodings.append(_sent_encodings)
        word_encodings.append(_word_encodings)
        mask_idxs.append(_mask_idxs)
    return sent_encodings, word_encodings, mask_idxs

  def make_predictions(self):
    output = []
    for q_idx, (w, s, m) in enumerate(zip(self.word_encodings, self.sent_encodings, self.mask_idxs)):
      print(f'Question {q_idx}')
      predictions = []
      candidate_input_ids = torch.stack([inp_ids['input_ids'].squeeze(0) for inp_ids in s])
      candidate_attention_masks = torch.stack([am['attention_mask'].squeeze(0) for am in s])
      candidate_logits = self.model(candidate_input_ids, attention_mask=candidate_attention_masks).logits
      for idx, (token, mask_idxs) in enumerate(zip(w, m)):
        mask_token_logits = candidate_logits[idx, mask_idxs, token]
        candidate_score = float(torch.mean(mask_token_logits))
        predictions.append(candidate_score)
      output.append(np.argmax(predictions) + 1)
    return output

  def get_model_accuracy(self, predictions, ground_truth):
    correct = 0
    for pred, gt in zip(predictions, ground_truth):
      if pred == gt:
        correct += 1
    return correct/len(ground_truth)



In [49]:
questions = pd.read_csv(os.path.join(mrscc_dir,"testing_data.csv"))
answers = pd.read_csv(os.path.join(mrscc_dir,"test_answer.csv"))
evaluator = LanguageModelEvaluator(questions, answers, choices, 'distilroberta-base', trainer.model)

1040


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [50]:
trainer.model.cpu()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [53]:
evaluator.run_model_and_evaluate()

Question 0
Question 1
Question 2
Question 3
Question 4
Question 5
Question 6
Question 7
Question 8
Question 9
Question 10
Question 11
Question 12
Question 13
Question 14
Question 15
Question 16
Question 17
Question 18
Question 19
Question 20
Question 21
Question 22
Question 23
Question 24
Question 25
Question 26
Question 27
Question 28
Question 29
Question 30
Question 31
Question 32
Question 33
Question 34
Question 35
Question 36
Question 37
Question 38
Question 39
Question 40
Question 41
Question 42
Question 43
Question 44
Question 45
Question 46
Question 47
Question 48
Question 49
Question 50
Question 51
Question 52
Question 53
Question 54
Question 55
Question 56
Question 57
Question 58
Question 59
Question 60
Question 61
Question 62
Question 63
Question 64
Question 65
Question 66
Question 67
Question 68
Question 69
Question 70
Question 71
Question 72
Question 73
Question 74
Question 75
Question 76
Question 77
Question 78
Question 79
Question 80
Question 81
Question 82
Question 83
Qu

In [54]:
evaluator.accuracy

0.7086538461538462